In [1]:
import pandas as pd                                                   
from tensorflow.keras.models import Sequential                        
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU               
from sklearn.preprocessing import MinMaxScaler                        
import numpy as np                                                    
import matplotlib.pyplot as plt                                        
import pickle
from scipy.stats import pearsonr
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.dataprocessing.transformers import MinTReconciliator

In [2]:
cal_day = []
cal_month = []
cal_month_non_zero = []
for i in range(1,32):
    if i < 10: 
        cal_month.append(str(i))
    elif i >= 10 and i <= 12:
        cal_month.append(str(i))  

In [3]:
GV = pd.read_csv("PPI_GV.csv")

Material_comp = pd.DataFrame()
Material_comp["time"] = GV["DATE"]
Material_comp = Material_comp.iloc[::-1]
Material_comp = Material_comp.iloc[:393]
Material_comp = Material_comp.iloc[::-1]
Material_comp = Material_comp.reset_index(drop=True)

GV = GV.drop(["DATE"], axis = 1) 
GV = GV.iloc[::-1]
GV = GV.iloc[:393]
GV = GV.iloc[::-1]
GV = GV.reset_index(drop=True)

CB = pd.read_csv("PPI_CB.csv")
CB = CB.drop(["DATE"], axis = 1) 
CB = CB.iloc[::-1]
CB = CB.iloc[:393]
CB = CB.iloc[::-1]
CB = CB.reset_index(drop=True) 

METAL = pd.read_csv("PPI_METAL.csv")
METAL = METAL.drop(["DATE"], axis = 1) 
METAL = METAL.iloc[::-1]
METAL = METAL.iloc[:393]
METAL = METAL.iloc[::-1]
METAL = METAL.reset_index(drop=True) 

CNC = pd.read_csv("PPI_CNC.csv")
CNC = CNC.drop(["DATE"], axis = 1) 
CNC = CNC.iloc[::-1]
CNC = CNC.iloc[:393]
CNC = CNC.iloc[::-1]
CNC = CNC.reset_index(drop=True) 

PL = pd.read_csv("PPI_PL.csv")
PL = PL.drop(["DATE"], axis = 1) 
PL = PL.iloc[::-1]
PL = PL.iloc[:393]
PL = PL.iloc[::-1]
PL = PL.reset_index(drop=True)

In [4]:
Material_comp["Gravel"] = GV
Material_comp["Conc_Brick"] = CB
Material_comp["Metals"] = METAL
Material_comp["Concrete_Mix"] = CNC
Material_comp["Plywood"] = PL

In [5]:
Material_comp

,time,Gravel,Conc_Brick,Metals,Concrete_Mix,Plywood
0,1990-01-01,123.700,122.2,119.5,110.300,114.300
1,1990-02-01,124.600,122.2,118.8,110.300,115.800
2,1990-03-01,125.100,122.2,118.8,110.900,116.100
3,1990-04-01,125.500,122.2,118.8,111.000,125.300
4,1990-05-01,126.000,122.2,118.8,111.300,122.700
...,...,...,...,...,...,...
388,2022-05-01,410.942,304.653,374.857,318.246,355.223
389,2022-06-01,421.041,309.714,378.454,322.084,347.477
390,2022-07-01,423.964,309.714,372.821,328.435,324.565
391,2022-08-01,426.322,309.714,367.714,332.472,316.151


In [8]:
scaler = Scaler()
series = scaler.fit_transform(TimeSeries.from_dataframe(
            Material_comp, "time", ["Gravel","Conc_Brick","Metals","Concrete_Mix","Plywood"])).astype(np.float32)
train, val = series.split_after(pd.Timestamp("20160401"))

In [10]:
model_materials =  NBEATSModel(input_chunk_length = 24,output_chunk_length=7, log_tensorboard=True)
model_materials.fit(train, val_series=val, verbose=True)

2022-11-01 20:46:54 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-11-01 20:46:54 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-11-01 20:46:54 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-11-01 20:46:54 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-11-01 20:46:54 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 7.0 M 
---------------------------------------------------
7.0 M     Trainable params
2.0 K     Non-trainable params
7.0 M     Total params
27.822    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Dingus-Elite\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\pytorch_lightning\trainer\trainer.py:1892: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

2022-11-01 20:48:47 pytorch_lightning.utilities.rank_zero INFO: `Trainer.fit` stopped: `max_epochs=100` reached.


In [15]:
pred = model_materials.predict(n=7,series=series)

Predicting: 9it [00:00, ?it/s]

In [16]:
scaled_pred=scaler.inverse_transform(pred)

In [17]:
pred_out = TimeSeries.to_csv(scaled_pred,"Mat_pred.csv")

In [18]:
model_materials.save("NBeats_model.pkl")

In [19]:
Material_comp.to_csv("Material_comp.csv")

PermissionError: [Errno 13] Permission denied: 'Material_comp.csv'

In [ ]:
df_pred = scaled_pred.pd_dataframe([])

In [ ]:
mat_list = df_pred.to_numpy()

In [ ]:
mat_list

In [ ]:
n = 7
score = np.empty([n,5])
for material in range(0,5):
    for month in range(0,n):
        if month < n-1:
            if  mat_list[month][material] <= mat_list[month+1][material]:
                score[month][material] = 1
            else:
                score[month][material] = 2
        else:
            if  mat_list[month][material] <= mat_list[month-1][material]:
                score[month][material] = 1
            else:
                score[month][material] = 2

score

In [ ]:
mat_score= np.sum(score, axis=0)
mat_score.tolist()

In [ ]:
sugg = []
for i in range(len(mat_score)):
    if mat_score[i] <= 7:
        sugg.append("Prices may be dropping in the next few months.")
    else: 
        sugg.append("Prices may be rising in the next few months.")

In [ ]:
sugg

In [ ]:
change_per = []
for i in range(0,5):
    change_per.append(str("{:.2f}".format(((mat_list[0][i]-mat_list[1][i])/mat_list[0][i])*100))+"%")

In [ ]:
change_per